In [2]:
import os
if not os.path.isdir('k'):
 !git clone -b development16 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
!cd k && pip install .

Cloning into 'k'...


^C


Processing c:\users\dddl distributed\documents\keffnet_learning\k
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pandas-2.0.1-cp39-cp39-win_amd64.whl (10.7 MB)
     ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
     --------------------------------------- 0.0/23.9 MB 495.5 kB/s eta 0:00:49
     --------------------------------------- 0.1/23.9 MB 469.7 kB/s eta 0:00:51
     --------------------------------------- 0.1/23.9 MB 525.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/23.9 MB 469.7 kB/s eta 0:00:51
     --------------------------------------- 0.1/23.9 MB 481.4 kB/s eta 0:00:50
     --------------------------------------- 0.2/23.9 MB 482.7 kB/s eta 0:00:50
     --------------------------------------- 0.2/23.9 MB 456.4 kB/s eta 0:

In [3]:
import cai.layers
import cai.datasets
import cai.efficientnet
import cai.util
import gc
import multiprocessing
import random
import tensorflow as tf
import numpy as np
#import matplotlib.pylab as plt
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.datasets
from tensorflow.keras import regularizers
import psutil

print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
print("CPU cores:", multiprocessing.cpu_count())
print('RAM:', (psutil.virtual_memory().total / 1e9),'GB')
print(tf.config.list_physical_devices('GPU'))

Tensorflow version: 2.10.1
Keras version: 2.10.0
CPU cores: 20
RAM: 137.24317696 GB
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
batch_size=32 # @param [16, 32, 64, 128, 256, 512] {type:"raw"} 
channels_per_group=16 # @param [2, 4, 8, 12, 16, 32] {type:"raw"}
epochs=50 # @param [2, 25, 50, 75, 100, 100, 200, 400] {type:"raw"}
verbose=True #@param {type:"boolean"}
bipolar_input=True #@param {type:"boolean"}
seed=7

input_shape = (32, 32, 1)

num_classes = 3

if channels_per_group==2:
   kTypes = [cai.layers.D6v3_2ch()]
elif channels_per_group==4:
   kTypes = [cai.layers.D6v3_4ch()]
elif channels_per_group==8:
   kTypes = [cai.layers.D6v3_8ch()]
elif channels_per_group==12:
   kTypes = [cai.layers.D6v3_12ch()]
elif channels_per_group==16:
   kTypes = [cai.layers.D6v3_16ch()]
else:
   kTypes = [cai.layers.D6v3_32ch()]

In [11]:
from sklearn.metrics import balanced_accuracy_score

In [ ]:
def work_on_efficientnetV2(show_model=False, run_fit=False, test_results=False, test_part, calc_f1=False, kTypes=[]):
  monitor='val_loss'
  if (show_model):
    input_shape = global_input_shape
  else:
    input_shape = (None, None, global_input_shape[2])
  for kType in kTypes:
      basefilename = 'kEffNetV2-'+str(kType) + str(batch_size)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      model = cai.efficientnet.kEfficientNetB0(
        include_top=True,
        skip_stride_cnt=3,
        input_shape=input_shape,
        classes=num_classes,
        kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=[balanced_accuracy_score])

      if (show_model): 
        model.summary(line_length=180)

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit): 
            #train_flow = train_datagen.flow(
            #    X_train, y_train,
            #    batch_size=batch_size,
            #    shuffle=True,
            #    seed=seed
            #)
            #validation_flow = valid_datagen.flow(
            #    X_val, y_val,
            #    batch_size=batch_size,
            #    shuffle=False,
            #    seed=seed
            #)
            history = model.fit(
              X_train, y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data= (X_val, y_val),
              callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
              workers=cpus_num,
              max_queue_size=128
            )
            plt.figure()
            plt.ylabel("Accuracy (training and validation)")
            plt.xlabel("Epochs")
            plt.ylim([0,1])
            plt.plot(history.history["accuracy"])
            plt.plot(history.history["val_accuracy"])
      if (test_results):
        #test_flow = test_datagen.flow(
        #    X_test, y_test,
        #    batch_size=batch_size,
        #    shuffle=True,
        #    seed=seed
        #)
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            X_test, y_test,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      #if (calc_f1):
        #cai.datasets.test_flips_on_saved_model(X_test, y_test, best_result_file_name, has_flip_x=True, has_flip_y=True, has_bw=False, center_crop=0.15)
      print('Finished: '+basefilename)


In [5]:
cpus_num = max([multiprocessing.cpu_count(), 8])

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [20]:
def balanced_accuracy(y_true, y_pred):
  acc = 0.5
  print("sdfsdf", y_pred)
  y_pred = np.argmax(y_pred, axis=1)
  print(y_pred)
  return acc

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
for i in range(7, 8):
    basefilename = 'kEffNetV2-'+str(i+1)
    best_result_file_name = basefilename+'-best_result.hdf5'
    print('Running: '+basefilename)
    model = cai.efficientnet.kEfficientNetB0(
    include_top=True,
    skip_stride_cnt=3,
    input_shape=input_shape,
    classes=num_classes,
    kType=kTypes[0])
    monitor = 'val_accuracy'
    optimizer = keras.optimizers.RMSprop()
    model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

    save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

    X_train = np.load("parts/X_tr" + str(i+1) + ".npy")
    y_train = np.load("parts/y_tr" + str(i+1) + ".npy")
    y_train = to_categorical(y_train, 3)
    y_train = y_train.reshape((y_train.shape[1], 3))
    X_val = np.load("parts/X_vl" + str(i+1) + ".npy")
    y_val = np.load("parts/y_vl" + str(i+1) + ".npy")
    y_val = to_categorical(y_val, 3)
    y_val = y_val.reshape((y_val.shape[1], 3))
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        class_weight={0:3.6, 1:1.26, 2:1},
        batch_size=batch_size,
        validation_data= (X_val, y_val),
        callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25), callback],
        workers=cpus_num,
        max_queue_size=128
    )

Running: kEffNetV2-8
Epoch 1/50
370/370 [==============================] - ETA: 0s - loss: 1.7425 - accuracy: 0.6653
Epoch 1: val_accuracy improved from -inf to 0.13431, saving model to kEffNetV2-8-best_result.hdf5
370/370 [==============================] - 112s 272ms/step - loss: 1.7425 - accuracy: 0.6653 - val_loss: 1.6042 - val_accuracy: 0.1343 - lr: 0.0010
Epoch 2/50
370/370 [==============================] - ETA: 0s - loss: 0.9274 - accuracy: 0.7208
Epoch 2: val_accuracy improved from 0.13431 to 0.74412, saving model to kEffNetV2-8-best_result.hdf5
370/370 [==============================] - 95s 256ms/step - loss: 0.9274 - accuracy: 0.7208 - val_loss: 0.6253 - val_accuracy: 0.7441 - lr: 0.0015
Epoch 3/50
370/370 [==============================] - ETA: 0s - loss: 0.8316 - accuracy: 0.7841
Epoch 3: val_accuracy did not improve from 0.74412
370/370 [==============================] - 94s 255ms/step - loss: 0.8316 - accuracy: 0.7841 - val_loss: 1.3582 - val_accuracy: 0.4588 - lr: 0.0020

In [14]:
acc_res = []
for i in range(10):
    basefilename = 'kEffNetV2-'+str(i+1)
    best_result_file_name = basefilename+'-best_result.hdf5'
    model = cai.models.load_kereas_model(best_result_file_name)
    X_test = np.load("parts/X_tst" + str(i+1) + '.npy')
    y_test = np.load("parts/y_tst" + str(i+1) + '.npy')
    y_test = to_categorical(y_test, 3)
    y_test = y_test.reshape((y_test.shape[1], 3))
    print(best_result_file_name)
    evaluated = model.evaluate(
        X_test, y_test,
        batch_size=batch_size,
        use_multiprocessing=False,
        workers=cpus_num
    )
    acc_res.append(evaluated[1])
    for metric, name in zip(evaluated,["loss","acc"]):
        print(name,metric)

kEffNetV2-1-best_result.hdf5
32/32 [==============================] - 4s 61ms/step - loss: 0.3914 - accuracy: 0.8451
loss 0.39136412739753723
acc 0.8450980186462402
kEffNetV2-2-best_result.hdf5
32/32 [==============================] - 4s 67ms/step - loss: 0.6577 - accuracy: 0.8504
loss 0.6576752066612244
acc 0.8504399061203003
kEffNetV2-3-best_result.hdf5
32/32 [==============================] - 4s 66ms/step - loss: 0.5348 - accuracy: 0.8599
loss 0.5348047614097595
acc 0.8599412441253662
kEffNetV2-4-best_result.hdf5
32/32 [==============================] - 3s 50ms/step - loss: 0.3813 - accuracy: 0.8613
loss 0.3812827169895172
acc 0.861328125
kEffNetV2-5-best_result.hdf5
32/32 [==============================] - 4s 68ms/step - loss: 0.2636 - accuracy: 0.8952
loss 0.2636420726776123
acc 0.895200788974762
kEffNetV2-6-best_result.hdf5
32/32 [==============================] - 4s 67ms/step - loss: 0.4600 - accuracy: 0.8661
loss 0.4599878787994385
acc 0.8660801649093628
kEffNetV2-7-best_result

In [17]:
sum(acc_res)/len(acc_res)

0.8513713955879212

In [32]:
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score

In [35]:
cf_m = []
for i in range(10):
    basefilename = 'kEffNetV2-'+str(i+1)
    best_result_file_name = basefilename+'-best_result.hdf5'
    model = cai.models.load_kereas_model(best_result_file_name)
    X_test = np.load("parts/X_tst" + str(i+1) + '.npy')
    y_test = np.load("parts/y_tst" + str(i+1) + '.npy')
    y_test = to_categorical(y_test, 3)
    y_test = y_test.reshape((y_test.shape[1], 3))
    predicted = np.argmax(model.predict(X_test), axis=-1)

    cf_matrix = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(model.predict(X_test), axis = -1))
    cf_m.append(cf_matrix)
    print(type(cf_matrix))

    
#ax = sns.heatmap(cf_matrix/cf_matrix.astype(np.float).sum(axis=1), annot=True, fmt='.2%', cmap='Blues')

32/32 [==============================] - 1s 43ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
32/32 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>
33/33 [==============================] - 1s 42ms/step
<class 'numpy.ndarray'>


In [63]:
for i in range(10):
    print(cf_m[i]/np.transpose(cf_m[i].astype(np.float).sum(axis=1)).reshape(3, 1)*100)

[[96.35036496  3.64963504  0.        ]
 [ 2.05128205 92.56410256  5.38461538]
 [ 8.31643002 16.8356998  74.84787018]]
[[89.85507246  2.17391304  7.97101449]
 [ 5.62659847 84.14322251 10.23017903]
 [ 0.         15.58704453 84.41295547]]
[[96.37681159  2.89855072  0.72463768]
 [ 0.76923077 89.23076923 10.        ]
 [ 9.93914807  9.53346856 80.52738337]]
[[100.           0.           0.        ]
 [  4.85933504  87.72378517   7.4168798 ]
 [  5.06072874  13.96761134  80.97165992]]
[[97.10144928  1.44927536  1.44927536]
 [ 0.51282051 87.17948718 12.30769231]
 [ 0.         10.7505071  89.2494929 ]]
[[93.47826087  0.72463768  5.79710145]
 [ 1.53452685 83.88746803 14.57800512]
 [ 0.         13.15789474 86.84210526]]
[[100.           0.           0.        ]
 [ 10.51282051  86.15384615   3.33333333]
 [  8.51926978  43.0020284   48.47870183]]
[[96.4028777   1.43884892  2.15827338]
 [ 0.25575448 92.07161125  7.67263427]
 [ 5.26315789 11.33603239 83.40080972]]
[[99.27007299  0.72992701  0.        ]

C:\Users\DDDL Distributed\AppData\Local\Temp\ipykernel_2664\132735487.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(cf_m[i]/np.transpose(cf_m[i].astype(np.float).sum(axis=1)).reshape(3, 1)*100)


In [60]:
cf_m[0].astype(np.float).sum(axis=1), cf_m[0], cf_m[0]/np.transpose(cf_m[0].astype(np.float).sum(axis=1)).reshape(3, 1)

C:\Users\DDDL Distributed\AppData\Local\Temp\ipykernel_2664\3583588078.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cf_m[0].astype(np.float).sum(axis=1), cf_m[0], cf_m[0]/np.transpose(cf_m[0].astype(np.float).sum(axis=1)).reshape(3, 1)


(array([137., 390., 493.]),
 array([[132,   5,   0],
        [  8, 361,  21],
        [ 41,  83, 369]], dtype=int64),
 array([[0.96350365, 0.03649635, 0.        ],
        [0.02051282, 0.92564103, 0.05384615],
        [0.0831643 , 0.168357  , 0.7484787 ]]))

In [66]:
for i, k in zip(cf_m, range(10)):
    np.save('cf_' + str(k), i)